In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet_v2 import preprocess_input 

# ======================
# 1. SETUP & VARIABLES
# ======================
print("Loading model... please wait.")
# Make sure the file name matches exactly what you saved earlier
model = load_model("asl_resnet50v2_optimized.h5") 

classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 
           'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 
           'W', 'X', 'Y', 'Z', 'nothing', 'space', 'del']

input_shape = (128, 128)

# Variables to hold text
current_word = ""
current_prediction = "..."

# ======================
# 2. MOUSE CALLBACK FUNCTION
# ======================
def mouse_click(event, x, y, flags, param):
    global current_word, current_prediction
    
    if event == cv2.EVENT_LBUTTONDOWN:
        # Check if "ADD LETTER" button is clicked
        if 660 < x < 840 and 250 < y < 300:
            if current_prediction not in ['nothing', 'del', 'space']:
                current_word += current_prediction
            elif current_prediction == 'space':
                current_word += " "
        
        # Check if "ADD SPACE" button is clicked
        elif 660 < x < 840 and 310 < y < 360:
            current_word += " "

        # Check if "BACKSPACE" button is clicked
        elif 660 < x < 840 and 370 < y < 420:
            current_word = current_word[:-1]

        # Check if "CLEAR ALL" button is clicked
        elif 660 < x < 840 and 430 < y < 480:
            current_word = ""

# ======================
# 3. MAIN LOOP
# ======================
# Open Webcam
cap = cv2.VideoCapture(0)

# Setup Window
cv2.namedWindow("ASL Translator")
cv2.setMouseCallback("ASL Translator", mouse_click)

print("Starting video stream... Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # FIX: Resize the frame to 640x480 so it fits our canvas perfectly
    frame = cv2.resize(frame, (640, 480))
    
    # Flip for mirror effect (natural interaction)
    frame = cv2.flip(frame, 1)
    
    # --- 1. DEFINE LAYOUT ---
    # Create a larger canvas: 480 height, 850 width (640 for cam + 210 for UI)
    canvas = np.zeros((480, 850, 3), dtype="uint8")
    canvas[:, :640] = frame # Paste webcam feed on the left
    
    # Draw UI Background (Dark Gray Panel on right)
    canvas[:, 640:] = (50, 50, 50) 
    
    # --- 2. PREDICTION LOGIC ---
    # Define ROI (Region of Interest) where hand should be
    x1, y1, x2, y2 = 100, 100, 400, 400
    roi = frame[y1:y2, x1:x2]
    
    # Draw ROI Box on feed (blue box)
    cv2.rectangle(canvas, (x1, y1), (x2, y2), (255, 0, 0), 2)
    
    if roi.size != 0:
        # Resize ROI to 128x128 for the ResNet model
        img = cv2.resize(roi, input_shape)
        img = img.astype("float32")
        
        # CRITICAL: Use ResNet specific preprocessing (-1 to 1 scaling)
        img = preprocess_input(img) 
        
        img = np.expand_dims(img, axis=0)
        
        # Predict
        pred = model.predict(img, verbose=0)
        class_idx = np.argmax(pred)
        current_prediction = classes[class_idx]
    
    # --- 3. DRAW UI ELEMENTS ---
    
    # [Prediction Display]
    cv2.putText(canvas, "Detected:", (650, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
    cv2.putText(canvas, current_prediction, (650, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 3)

    # [The Sentence Box]
    cv2.putText(canvas, "Word/Sentence:", (650, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
    cv2.rectangle(canvas, (650, 160), (840, 220), (255, 255, 255), -1) 
    display_text = current_word[-10:] # Show last 10 chars so it doesn't overflow
    cv2.putText(canvas, display_text, (655, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)

    # [Button 1: Add Letter (Green)]
    cv2.rectangle(canvas, (660, 250), (840, 300), (0, 200, 0), -1)
    cv2.putText(canvas, "ADD LETTER", (675, 285), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    # [Button 2: Add Space (Blue)]
    cv2.rectangle(canvas, (660, 310), (840, 360), (200, 100, 0), -1)
    cv2.putText(canvas, "ADD SPACE", (680, 345), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # [Button 3: Backspace (Orange)]
    cv2.rectangle(canvas, (660, 370), (840, 420), (0, 100, 255), -1)
    cv2.putText(canvas, "BACKSPACE", (680, 405), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # [Button 4: Clear (Red)]
    cv2.rectangle(canvas, (660, 430), (840, 480), (0, 0, 255), -1)
    cv2.putText(canvas, "CLEAR ALL", (690, 465), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # --- 4. DISPLAY ---
    cv2.imshow("ASL Translator", canvas)
    
    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()